<a href="https://colab.research.google.com/github/San-dra/Digital-Marketing-Performance-Dashboard-UrbanNest/blob/main/04_backfill_facebook_paid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Backfill Facebook Paid Ads Data (2022 to Today)
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

from google.auth import default
from googleapiclient.discovery import build
import gspread
import gspread_dataframe as gd
from google.colab import auth, userdata

In [ ]:
# Authenticate
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
drive_service = build('drive', 'v3')

In [ ]:
# SETTINGS
start_date = datetime(2022, 1, 1)
end_date = datetime.today()
folder_id = userdata.get('FACEBOOK_DATA_FOLDER')
file_name = f"facebook_paid_all_time_{datetime.today().strftime('%Y%m%d')}"



In [ ]:
# Load Campaigns Sheet
campaigns_spreadsheet = gc.open_by_key(userdata.get('CAMPAIGNS_SHEET_ID'))
df_campaigns = gd.get_as_dataframe(campaigns_spreadsheet.sheet1).dropna()

In [ ]:
# Function: Get active campaigns on a given day
def find_active_campaigns(date):
    return [camp for _, camp in df_campaigns.iterrows() if pd.to_datetime(camp['start_date']) <= date <= pd.to_datetime(camp['end_date'])]



In [ ]:
# Weights
country_weights = (
    ['Germany']*20 + ['France']*18 + ['Italy']*15 + ['Spain']*15 + ['Netherlands']*8 + ['Poland']*8 +
    ['Sweden']*5 + ['Ireland']*5 + ['Denmark']*4 + ['Belgium']*4 + ['Austria']*3 + ['Finland']*3 +
    ['Portugal']*3 + ['Czech Republic']*3 + ['Greece']*2 + ['Hungary']*2 + ['Romania']*2 +
    ['Bulgaria']*1 + ['Croatia']*1 + ['Cyprus']*1 + ['Estonia']*1 + ['Latvia']*1 + ['Lithuania']*1 +
    ['Luxembourg']*1 + ['Malta']*1 + ['Slovakia']*1 + ['Slovenia']*1
)

age_weights = ['18-24']*15 + ['25-34']*40 + ['35-44']*30 + ['45-54']*15
gender_weights = ['female']*60 + ['male']*38 + ['other']*2
device_weights = ['mobile']*60 + ['desktop']*35 + ['tablet']*5

ad_names = ["Spring Refresh", "Smart Storage", "Cozy Corners", "Scandi Style", "Balcony Ready", "Back Support Deals"]



In [ ]:
all_rows = []
current_date = start_date

while current_date <= end_date:
    active_campaigns = find_active_campaigns(current_date)

    for camp in active_campaigns:
        for _ in range(random.randint(2, 4)):
            country = random.choice(country_weights)
            ad_name = random.choice(ad_names)

            # Apply uplift for standout ads
            if ad_name in ["Smart Storage", "Scandi Style"]:
                impression_range = (5000, 15000)
                click_multiplier = (0.08, 0.12)
                spend_range = (150, 400)
                revenue_factor = (150, 300)
            else:
                impression_range = (1000, 10000)
                click_multiplier = (0.04, 0.08)
                spend_range = (50, 300)
                revenue_factor = (40, 250)

            impressions = random.randint(*impression_range)
            clicks = random.randint(20, int(impressions * random.uniform(*click_multiplier)))
            spend = round(random.uniform(*spend_range), 2)
            conversions = random.randint(0, int(clicks * 0.25))
            revenue = round(conversions * random.uniform(*revenue_factor), 2)
            ctr = round((clicks / impressions) * 100, 2)

            all_rows.append({
                'ad_id': f"fbp_{random.randint(100000,999999)}",
                'campaign_id': camp['campaign_id'],
                'ad_name': ad_name,
                'country': country,
                'age_group': random.choice(age_weights),
                'gender': random.choice(gender_weights),
                'device_category': random.choice(device_weights),
                'impressions': impressions,
                'clicks': clicks,
                'ctr_percent': ctr,
                'spend_usd': spend,
                'conversions': conversions,
                'revenue_usd': revenue,
                'date': current_date.strftime('%Y-%m-%d')
            })

    current_date += timedelta(days=1)

In [ ]:
# Upload to Google Sheets
df_all = pd.DataFrame(all_rows)

new_sheet = gc.create(file_name)
drive_service.files().update(
    fileId=new_sheet.id,
    addParents=folder_id,
    removeParents='root',
    fields='id, parents'
).execute()
worksheet = new_sheet.sheet1
gd.set_with_dataframe(worksheet, df_all)
print(f"Created new file '{file_name}' with {len(df_all)} rows.")

Created new file 'facebook_paid_all_time_20250430' with 2032 rows.
